In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.4 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 15.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 45.4 MB/s eta 0:00:0000:01


In [3]:
import pennylane as qml
from pennylane import numpy as np

In [5]:
dev = qml.device('default.qubit', wires=2)

x = np.array([0.1, 0.2, 0.3])

@qml.qnode(dev, diff_method="adjoint")
def circuit(a):
    qml.RX(a[0], wires=0)
    qml.CNOT(wires=(0,1))
    qml.RY(a[1], wires=1)
    qml.RZ(a[2], wires=1)
    return qml.expval(qml.PauliX(wires=1))

In [6]:
n_gates = 4
n_params = 3

ops = [
    qml.RX(x[0], wires=0),
    qml.CNOT(wires=(0,1)),
    qml.RY(x[1], wires=1),
    qml.RZ(x[2], wires=1)
]
M = qml.PauliX(wires=1)

In [7]:
state = dev._create_basis_state(0)

for op in ops:
    state = dev._apply_operation(state, op)

print(state)

[[9.82601808e-01-0.14850574j 9.85890302e-02+0.01490027j]
 [7.45635195e-04+0.00493356j 7.43148086e-03-0.04917107j]]


In [8]:
bra = dev._apply_operation(state, M)
ket = state

In [9]:
M_expval = np.vdot(bra, ket)
print("vdot  : ", M_expval)
print("QNode : ", circuit(x))

vdot  :  (0.18884787122715618+0j)
QNode :  0.18884787122715618


In [10]:
bra_n = dev._create_basis_state(0)

for op in ops:
    bra_n = dev._apply_operation(bra_n, op)
bra_n = dev._apply_operation(bra_n, M)
bra_n = dev._apply_operation(bra_n, qml.adjoint(ops[-1]))

ket_n = dev._create_basis_state(0)

for op in ops[:-1]: # don't apply last operation
    ket_n = dev._apply_operation(ket_n, op)

M_expval_n = np.vdot(bra_n, ket_n)
print(M_expval_n)

(0.18884787122715616+3.4558944247975454e-18j)


In [11]:
#Version 2
bra_n_v2 = dev._apply_operation(state, M)
ket_n_v2 = state

adj_op = qml.adjoint(ops[-1])

bra_n_v2 = dev._apply_operation(bra_n_v2, adj_op)
ket_n_v2 = dev._apply_operation(ket_n_v2, adj_op)

M_expval_n_v2 = np.vdot(bra_n_v2, ket_n_v2)
print(M_expval_n_v2)

(0.18884787122715616+1.6252868755895187e-18j)


In [12]:
bra_loop = dev._apply_operation(state, M)
ket_loop = state

for op in reversed(ops):
    adj_op = qml.adjoint(op)
    bra_loop = dev._apply_operation(bra_loop, adj_op)
    ket_loop = dev._apply_operation(ket_loop, adj_op)
    print(np.vdot(bra_loop, ket_loop))

(0.18884787122715616+1.6252868755895187e-18j)
(0.18884787122715618+5.718281173551752e-18j)
(0.18884787122715618+5.718281173551752e-18j)
(0.18884787122715618+5.718281173551752e-18j)


# ***Finding the Derivatives***

In [13]:
grad_op0 = qml.operation.operation_derivative(ops[0])
print(grad_op0)

[[-0.02498958+0.j          0.        -0.49937513j]
 [ 0.        -0.49937513j -0.02498958+0.j        ]]


In [14]:
bra = dev._apply_operation(state, M)
ket = state

grads = []

for op in reversed(ops):
    adj_op = qml.adjoint(op)
    ket = dev._apply_operation(ket, adj_op)

    # Calculating the derivative
    if op.num_params != 0:
        dU = qml.operation.operation_derivative(op)

        ket_temp = dev._apply_unitary(ket, dU, op.wires)

        dM = 2 * np.real(np.vdot(bra, ket_temp))
        grads.append(dM)

    bra = dev._apply_operation(bra, adj_op)


# Finally reverse the order of the gradients
# since we calculated them in reverse
grads = grads[::-1]

print("our calculation: ", grads)

grad_compare = qml.grad(circuit)(x)
print("comparison: ", grad_compare)

our calculation:  [-0.018947989233612107, 0.9316157966884513, -0.05841749223216956]
comparison:  [-0.01894799  0.9316158  -0.05841749]


***Same calculations using inbuild library diff_method="adjoint"***

In [15]:
dev_lightning = qml.device('lightning.qubit', wires=2)

@qml.qnode(dev_lightning, diff_method="adjoint")
def circuit_adjoint(a):
    qml.RX(a[0], wires=0)
    qml.CNOT(wires=(0,1))
    qml.RY(a[1], wires=1)
    qml.RZ(a[2], wires=1)
    return qml.expval(M)

qml.grad(circuit_adjoint)(x)

array([-0.01894799,  0.9316158 , -0.05841749])